In [1]:
import sqlite3
import os
import pandas as pd
import datetime
from datetime import date
from numpy import NaN

connexion = sqlite3.connect("input_data/bce.db")
cursor = connexion.cursor()

## Import all needed tables
The data come from fgov
voir documentation : https://economie.fgov.be/sites/default/files/Files/Entreprises/BCE/Cookbook-BCE-Open-Data.pdf
documentation 2: https://economie.fgov.be/sites/default/files/Files/Entreprises/BCE/Manuel-technique-reutilisation-des-donnees-BCE.pdf

In [2]:
cursor = cursor.execute( 
    """
    SELECT Category
    from code
    Group by Category
    """
)
result = cursor.fetchall()
col = [description[0] for description in cursor.description]
code = pd.DataFrame.from_records(data=result, columns=col)
category = list(code['Category'].T)

In [3]:
code_tbls = {}
for table_name in category:
    cursor.execute( f"""
        Select  Code, Description
        from code
        where code.Category = "{table_name}" AND code."Language"="FR";
        """
    )
    result = cursor.fetchall()
    col = [description[0] for description in cursor.description]
    code_tbls[table_name] = pd.DataFrame.from_records(data=result, columns=col)


In [4]:
cursor.execute("""
Select *
FROM enterprise
""")
enterprise = cursor.fetchall()
col = [description[0] for description in cursor.description]
enterprise = pd.DataFrame.from_records(data=enterprise, columns=col)


In [5]:
cursor.execute("""
Select *
FROM establishment
""")
establishment = cursor.fetchall()
col = [description[0] for description in cursor.description]
establishment = pd.DataFrame.from_records(data=establishment, columns=col)

In [6]:
dict_tbl = {}
list_key = [ 'JuridicalSituation', 'JuridicalForm', 'TypeOfEnterprise']

for key in list_key:
     dict_tbl[key] = code_tbls[key]

for tbls_name, tbls in dict_tbl.items():
    enterprise = enterprise.set_index(tbls_name).join(tbls.set_index('Code'), how='left')\
         .rename(columns={'Description' : tbls_name})

In [7]:
code_tbls['Nace2008']

,Code,Description
0,01,"Culture et production animale, chasse et servi..."
1,011,Cultures non permanentes
2,0111,"Culture de céréales (à l'exception du riz), de..."
3,01110,"Culture de céréales (à l'exception du riz), de..."
4,0111001,Culture de céréales (à l'exception du riz) : b...
...,...,...
3319,Q,SECTION Q -- SANTÉ HUMAINE ET ACTION SOCIALE
3320,R,"SECTION R -- ARTS, SPECTACLES ET ACTIVITÉS RÉC..."
3321,S,SECTION S -- AUTRES ACTIVITÉS DE SERVICES
3322,T,SECTION T -- ACTIVITÉS DES MÉNAGES EN TANT QU'...


## QUestion 1 what is the repartition of the companies Status ?
All entreprises are Active as non actives are removed from the list.
Further analysis could be done on a time dimention with a define time laps to see the number of compnies that whent from active to unactive.
see if data available otherwise when data update see what are the removed companies


In [8]:
enterprise

,EnterpriseNumber,Status,StartDate,JuridicalSituation,JuridicalForm,TypeOfEnterprise
1,0500.017.677,AC,2016-01-01 00:00:00,Situation normale,NaN,Personne physique
1,0500.044.797,AC,1800-01-01 00:00:00,Situation normale,NaN,Personne physique
1,0500.081.916,AC,1971-01-01 00:00:00,Situation normale,NaN,Personne physique
1,0500.083.993,AC,1965-01-01 00:00:00,Situation normale,NaN,Personne physique
1,0500.085.082,AC,2021-04-01 00:00:00,Situation normale,NaN,Personne physique
...,...,...,...,...,...,...
2,0859.371.993,AC,1986-07-01 00:00:00,Situation normale,Forme légale inconnue (ONSS),Personne morale
2,0859.372.092,AC,2000-07-01 00:00:00,Situation normale,Forme légale inconnue (ONSS),Personne morale
2,0859.380.804,AC,1982-07-01 00:00:00,Situation normale,Forme légale inconnue (ONSS),Personne morale
2,0850.392.169,AC,1970-01-01 00:00:00,Dissolution de plein droit par arrivée du terme,Forme légale inconnue (ONSS),Personne morale


## Question 2 juridical Form / situation

In [9]:
JurSit_repartition = (enterprise.groupby('JuridicalSituation')['EnterpriseNumber']
                .count()
                .reset_index(name='nmb_JurSit') )

# creation of a list of names with the goal to see it distribution 
JuridicalSituation_names = list(JurSit_repartition['JuridicalSituation'].T)

#repartition per category in %
JurSit_repartition['%']=JurSit_repartition['nmb_JurSit']/enterprise['EnterpriseNumber'].count()*100

#remove the other en create a separate frame
JurSit_repartition.loc[JurSit_repartition['JuridicalSituation'].isin(['Dissolution anticipée - Liquidation (dissolution volontaire)','Ouverture de faillite','Situation normale']),'cleaned_JuridicalSituation']= JurSit_repartition['JuridicalSituation']
JurSit_repartition.loc[~JurSit_repartition['JuridicalSituation'].isin(['Dissolution anticipée - Liquidation (dissolution volontaire)','Ouverture de faillite','Situation normale']),'cleaned_JuridicalSituation']= 'other'

#select only values that is = other
Fig_JuridicalSituation_others = JurSit_repartition.loc[JurSit_repartition['cleaned_JuridicalSituation']=='other']

# fig of values per juridiction
Fig_JuridicalSituation = JurSit_repartition.groupby('cleaned_JuridicalSituation')['%'].sum().reset_index(name=('%'))



In [10]:
Fig_JuridicalSituation

,cleaned_JuridicalSituation,%
0,Dissolution anticipée - Liquidation (dissoluti...,1.386095
1,Ouverture de faillite,4.010751
2,Situation normale,94.210784
3,other,0.392370


In [11]:
JurForm_repartition = (enterprise.groupby('JuridicalForm')['EnterpriseNumber']
                .count()
                .reset_index(name='nmb_JurForm') )


#repartition per category in %
JurForm_repartition['%']=JurForm_repartition['nmb_JurForm']/enterprise['EnterpriseNumber'].count()*100

JurForm_repartition = JurForm_repartition.sort_values(by=['%'],ascending=False ).head(20)
# creation of a list of names with the goal to see it distribution 
JuridicalForm_names = list(JurForm_repartition['JuridicalForm'].T)

#remove the other en create a separate frame
selection = ['Société privée à responsabilité limitée',
            'Société à responsabilité limitée',
            'Association sans but lucratif',
            'Société anonyme',
            'Association des copropriétaires',
            'Entité étrangère']

JurForm_repartition.loc[JurForm_repartition['JuridicalForm'].isin(selection),'cleaned_JuridicalForm']= JurForm_repartition['JuridicalForm']
JurForm_repartition.loc[~JurForm_repartition['JuridicalForm'].isin(selection),'cleaned_JuridicalForm']= 'other'

#select only values that is = other
Fig_JuridicalForm_others = JurForm_repartition.loc[JurForm_repartition['cleaned_JuridicalForm']=='other']

# fig of values per juridiction
Fig_JuridicalForm = JurForm_repartition.groupby('cleaned_JuridicalForm')['%'].sum().reset_index(name=('%'))

In [12]:
Fig_JuridicalForm

,cleaned_JuridicalForm,%
0,Association des copropriétaires,5.045640
1,Association sans but lucratif,7.533709
2,Entité étrangère,3.320736
3,Société anonyme,5.315055
4,Société privée à responsabilité limitée,19.133287
5,Société à responsabilité limitée,9.156214
6,other,8.764327


In [13]:
#select only values that is = other
Fig_JuridicalSituation_others = JurSit_repartition.loc[JurSit_repartition['cleaned_JuridicalSituation']=='other']
# fig of values per juridiction
Fig_JuridicalSituation = JurSit_repartition.groupby('cleaned_JuridicalSituation')['%'].sum()

## Question 3 Which percentage of the companies are which type of entreprise?

In [14]:
Types_of_companies = (enterprise.groupby('TypeOfEnterprise')['EnterpriseNumber']
                .count()
                .reset_index(name='nmb_JurSit') )
Types_of_companies['%']=Types_of_companies['nmb_JurSit']/enterprise['EnterpriseNumber'].count()*100
Types_of_companies

,TypeOfEnterprise,nmb_JurSit,%
0,Personne morale,1099649,59.016461
1,Personne physique,763643,40.983539


## Question 4 What is the average company's age in each sector (hint: look what is the NACE code)?

In [15]:
#define the age of the company
today_date = datetime.datetime.now().date()
enterprise['StartDate']= pd.to_datetime(enterprise['StartDate']).dt.date
enterprise['Age']= (today_date-enterprise['StartDate'])
enterprise['Age'] = (enterprise['Age'].dt.days/365).round()

In [16]:
enterprise

,EnterpriseNumber,Status,StartDate,JuridicalSituation,JuridicalForm,TypeOfEnterprise,Age
1,0500.017.677,AC,2016-01-01,Situation normale,NaN,Personne physique,7.0
1,0500.044.797,AC,1800-01-01,Situation normale,NaN,Personne physique,223.0
1,0500.081.916,AC,1971-01-01,Situation normale,NaN,Personne physique,52.0
1,0500.083.993,AC,1965-01-01,Situation normale,NaN,Personne physique,58.0
1,0500.085.082,AC,2021-04-01,Situation normale,NaN,Personne physique,2.0
...,...,...,...,...,...,...,...
2,0859.371.993,AC,1986-07-01,Situation normale,Forme légale inconnue (ONSS),Personne morale,36.0
2,0859.372.092,AC,2000-07-01,Situation normale,Forme légale inconnue (ONSS),Personne morale,22.0
2,0859.380.804,AC,1982-07-01,Situation normale,Forme légale inconnue (ONSS),Personne morale,40.0
2,0850.392.169,AC,1970-01-01,Dissolution de plein droit par arrivée du terme,Forme légale inconnue (ONSS),Personne morale,53.0


In [17]:
Query_activity = """
    SELECT EntityNumber, substr(NaceCode,1,2) Industry, NaceVersion, ActivityGroup, Classification
    FROM activity
    GROUP BY EntityNumber , Industry, NaceVersion, ActivityGroup, Classification
"""

cursor.execute(Query_activity)
activity = cursor.fetchall()
col = [description[0] for description in cursor.description]

activity = pd.DataFrame.from_records(data=activity, columns=col)

In [18]:
activity

,EntityNumber,Industry,NaceVersion,ActivityGroup,Classification
0,0200.065.765,41,2008,001,MAIN
1,0200.065.765,70,2003,001,MAIN
2,0200.065.765,84,2008,006,MAIN
3,0200.068.636,36,2008,001,MAIN
4,0200.068.636,36,2008,006,MAIN
...,...,...,...,...,...
9567424,2.331.747.168,74,2008,003,MAIN
9567425,2.331.747.168,85,2008,003,MAIN
9567426,2.331.747.168,86,2008,003,MAIN
9567427,2.331.747.168,96,2008,003,MAIN


### Check the not na when merging entreprise number in entreprise en activity

In [19]:
activity_EntNum = activity['EntityNumber'].drop_duplicates()

#Check ENtreprise nbr without activity linked
a = enterprise['EnterpriseNumber']
b = activity_EntNum
left_a_b = pd.merge(a,b,left_on='EnterpriseNumber', right_on='EntityNumber',how='left')
Left_a_b_nona = left_a_b[left_a_b['EntityNumber'].notna()]

#Check ENtreprise nbr without activity linked
left_a_b_Na = left_a_b[left_a_b['EntityNumber'].isna()]
Ent_withoutactiv = pd.merge(left_a_b_Na,enterprise,on='EnterpriseNumber',how='left')
Ent_withoutactiv

,EnterpriseNumber,EntityNumber,Status,StartDate,JuridicalSituation,JuridicalForm,TypeOfEnterprise,Age
0,0500.044.797,NaN,AC,1800-01-01,Situation normale,NaN,Personne physique,223.0
1,0500.083.993,NaN,AC,1965-01-01,Situation normale,NaN,Personne physique,58.0
2,0500.122.201,NaN,AC,1969-11-01,Situation normale,NaN,Personne physique,53.0
3,0500.153.081,NaN,AC,1973-03-01,Situation normale,NaN,Personne physique,50.0
4,0500.181.785,NaN,AC,1975-08-01,Situation normale,NaN,Personne physique,47.0
...,...,...,...,...,...,...,...,...
683112,0859.371.993,NaN,AC,1986-07-01,Situation normale,Forme légale inconnue (ONSS),Personne morale,36.0
683113,0859.372.092,NaN,AC,2000-07-01,Situation normale,Forme légale inconnue (ONSS),Personne morale,22.0
683114,0859.380.804,NaN,AC,1982-07-01,Situation normale,Forme légale inconnue (ONSS),Personne morale,40.0
683115,0850.392.169,NaN,AC,1970-01-01,Dissolution de plein droit par arrivée du terme,Forme légale inconnue (ONSS),Personne morale,53.0


In [20]:
a = enterprise
b = activity
inner_activity_entreprise = pd.merge(a,b,left_on='EnterpriseNumber', right_on='EntityNumber',how='left')


In [24]:
entreprise_activities = inner_activity_entreprise[inner_activity_entreprise['Classification']\
                                                .isin(['MAIN',NaN])]\
                                                .sort_values(by=['EnterpriseNumber','ActivityGroup','NaceVersion'])\
                                                .drop_duplicates(subset=['EnterpriseNumber','ActivityGroup'],keep="last")

In [34]:
a = entreprise_activities[entreprise_activities['NaceVersion']=='2008']
b = code_tbls['Nace2008']
Nace2008 = pd.merge(a,b,left_on='Industry', right_on='Code',how='left')

a = entreprise_activities[entreprise_activities['NaceVersion']=='2003']
b = code_tbls['Nace2003']
Nace2003 = pd.merge(a,b,left_on='Industry', right_on='Code',how='left')

a = entreprise_activities[entreprise_activities['NaceVersion'].isna()]
b = code_tbls['Nace2003']
NaceNaN= pd.merge(a,b,left_on='Industry', right_on='Code',how='left')

entreprise_activities = pd.concat([Nace2008, Nace2003, NaceNaN])
entreprise_activities


,EnterpriseNumber,Status,StartDate,JuridicalSituation,JuridicalForm,TypeOfEnterprise,Age,EntityNumber,Industry,NaceVersion,ActivityGroup,Classification,Code,Description
0,0200.065.765,AC,1960-08-09,Situation normale,Association prestataire de services (Région fl...,Personne morale,62.0,0200.065.765,41,2008,001,MAIN,41,Construction de bâtiments; promotion immobilière
1,0200.065.765,AC,1960-08-09,Situation normale,Association prestataire de services (Région fl...,Personne morale,62.0,0200.065.765,84,2008,006,MAIN,84,Administration publique et défense; sécurité s...
2,0200.068.636,AC,1923-02-16,Situation normale,Association chargée de mission (Région flamande),Personne morale,100.0,0200.068.636,36,2008,001,MAIN,36,"Captage, traitement et distribution d'eau"
3,0200.068.636,AC,1923-02-16,Situation normale,Association chargée de mission (Région flamande),Personne morale,100.0,0200.068.636,36,2008,006,MAIN,36,"Captage, traitement et distribution d'eau"
4,0200.305.493,AC,1922-01-01,Situation normale,Association prestataire de services (Région fl...,Personne morale,101.0,0200.305.493,94,2008,001,MAIN,94,Activités des organisations associatives
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683112,0998.603.023,AC,1992-01-01,Situation normale,Entité étrangère avec un bien immobilier en Be...,Personne morale,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
683113,0998.603.518,AC,2001-01-01,Situation normale,Entité étrangère avec un bien immobilier en Be...,Personne morale,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
683114,0998.603.617,AC,2001-01-01,Situation normale,Entité étrangère avec un bien immobilier en Be...,Personne morale,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
683115,0998.604.211,AC,2001-09-15,Situation normale,Entité étrangère avec un bien immobilier en Be...,Personne morale,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
entreprise_activities[entreprise_activities['StartDate']!=pd.to_datetime('1800-01-01')]

/tmp/ipykernel_18435/2911278602.py:1: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  entreprise_activities[entreprise_activities['StartDate']!=pd.to_datetime('1800-01-01')]


,EnterpriseNumber,Status,StartDate,JuridicalSituation,JuridicalForm,TypeOfEnterprise,Age,EntityNumber,Industry,NaceVersion,ActivityGroup,Classification,Code,Description
0,0200.065.765,AC,1960-08-09,Situation normale,Association prestataire de services (Région fl...,Personne morale,62.0,0200.065.765,41,2008,001,MAIN,41,Construction de bâtiments; promotion immobilière
1,0200.065.765,AC,1960-08-09,Situation normale,Association prestataire de services (Région fl...,Personne morale,62.0,0200.065.765,84,2008,006,MAIN,84,Administration publique et défense; sécurité s...
2,0200.068.636,AC,1923-02-16,Situation normale,Association chargée de mission (Région flamande),Personne morale,100.0,0200.068.636,36,2008,001,MAIN,36,"Captage, traitement et distribution d'eau"
3,0200.068.636,AC,1923-02-16,Situation normale,Association chargée de mission (Région flamande),Personne morale,100.0,0200.068.636,36,2008,006,MAIN,36,"Captage, traitement et distribution d'eau"
4,0200.305.493,AC,1922-01-01,Situation normale,Association prestataire de services (Région fl...,Personne morale,101.0,0200.305.493,94,2008,001,MAIN,94,Activités des organisations associatives
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683112,0998.603.023,AC,1992-01-01,Situation normale,Entité étrangère avec un bien immobilier en Be...,Personne morale,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
683113,0998.603.518,AC,2001-01-01,Situation normale,Entité étrangère avec un bien immobilier en Be...,Personne morale,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
683114,0998.603.617,AC,2001-01-01,Situation normale,Entité étrangère avec un bien immobilier en Be...,Personne morale,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
683115,0998.604.211,AC,2001-09-15,Situation normale,Entité étrangère avec un bien immobilier en Be...,Personne morale,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Regarding the Entreprise without Activities ??
further analysis has been made to see if establishement could fill the missing entreprise number but every establishement as a entreprise number in the table entreprise. (except 4 observation). see here under SQL command 

Select count(EstablishmentNumber)
FROM establishment
INNER JOIN (
	Select EnterpriseNumber as r_EnterpriseNumber
	FROM enterprise
	)
ON EnterpriseNumber = r_EnterpriseNumber
WHERE r_EnterpriseNumber NOTNULL  

Entreprises without an activity 683117


In [22]:
Ent_withoutactiv.groupby(['JuridicalForm','TypeOfEnterprise'])['EnterpriseNumber'].count().order

AttributeError: 'Series' object has no attribute 'order'

In [ ]:
code_tbls['Nace2008']

,Code,Description
0,01,"Culture et production animale, chasse et servi..."
1,011,Cultures non permanentes
2,0111,"Culture de céréales (à l'exception du riz), de..."
3,01110,"Culture de céréales (à l'exception du riz), de..."
4,0111001,Culture de céréales (à l'exception du riz) : b...
...,...,...
3319,Q,SECTION Q -- SANTÉ HUMAINE ET ACTION SOCIALE
3320,R,"SECTION R -- ARTS, SPECTACLES ET ACTIVITÉS RÉC..."
3321,S,SECTION S -- AUTRES ACTIVITÉS DE SERVICES
3322,T,SECTION T -- ACTIVITÉS DES MÉNAGES EN TANT QU'...


In [ ]:
code_tbls['ActivityGroup']

,Code,Description
0,001,Activités TVA
1,002,Activités EDRL
2,003,Activités
3,004,Activités de la fonction publique fédérale
4,005,Activités ONSSAPL
5,006,Activités ONSS
6,007,Activités de l'enseignement subventionné
